In [1]:
from sage.rings.finite_rings.hom_finite_field import FiniteFieldHomomorphism_generic

In [2]:
def get_defining_set(n, pol, field):
    defining_set = []
    
    while len(defining_set) != n:
        aux = field.random_element()
        if pol(aux) != 0 and aux not in defining_set:
            defining_set = defining_set + [aux]
    
    return defining_set

In [3]:
from sage.coding.linear_code import AbstractLinearCode
from sage.coding.encoder import Encoder
from sage.coding.decoder import Decoder

class Goppa(AbstractLinearCode):
    r"""
    Implementation of Goppa codes.
    
    INPUT:
    - ``field`` -- finite field on which `self` is defined.
    - ``generating_pol`` -- a monic polynomial with coefficients in
    a finite field `\GF{p^m}` extending from `field`.
    
    - ``defining_set`` -- tuple of n distinct elements of `\GF{p^m}`
    that are not roots of `generating_pol`
    """
    _registered_decoders = {}
    def __init__(self, defining_set, generating_pol, field):
        """
        Initialize.
        """
        # TODO: comprobar que el mcd sea 1
        
        if not generating_pol.is_monic():
            raise ValueError("ERROR. Generating polynomial isn't monic")
        
        for gamma in defining_set:
            if generating_pol(gamma) == 0:
                raise ValueError("ERROR. Defining elements are roots of generating polynomial")
        
        self._field = field
        self._field_L = generating_pol.base_ring()

        if not self._field.is_field() or not self._field.is_finite():
            raise ValueError("ERROR. Generating polynomial isn't definied over a finite field")
        
        self._length = len(defining_set)
        self._generating_pol = generating_pol
        self._defining_set = defining_set
        
        super(Goppa, self).__init__(self._field, self._length, "GoppaEncoder", "GoppaDecoder")
        
        if self.get_dimension() == 0:
            raise ValueError("ERROR. Code dimension is null")
    
    def _repr_(self):
        """
        Representation of a Goppa code.
        """
        return "[{}, {}] Goppa code".format(self._length, self.get_dimension())
    
    def get_generating_pol(self):
        """
        Return the generating polynomial of ``self``.
        """ 
        return self._generating_pol
    
    def get_defining_set(self):
        """
        Return the defining set of ``self``.
        """ 
        return self._defining_set
    
    def get_parity_pol(self):
        """
        Return the parity polynomial of ``self``.
        """
        parity_pol = list()
        
        for elem in self._defining_set:
            parity_pol.append((self._generating_pol.parent().gen() - elem).inverse_mod(self._generating_pol))
        
        return parity_pol
    
    def get_parity_check_matrix(self):
        """
        Return a parity check matrix of ``self``.
        """
        V, from_V, to_V = self._field_L.vector_space(self._field, map = True)

        parity = self.get_parity_pol()

        vector_L = []
        for i in range(len(parity)):
            vector_L = vector_L + parity[i].list()

        vector_F = []
        for i in range(len(vector_L)):
            vector_F = vector_F + to_V(vector_L[i]).list()

        matriz_F = matrix(self._length, vector_F)
        matriz_F = matriz_F.T
        
        return matriz_F
    
    def get_generator_matrix(self):
        """
        Return a generador matrix of ``self``.
        """
        H = self.get_parity_check_matrix()
        G = transpose(H).left_kernel().basis_matrix()
        
        return G
    
    def get_dimension(self):
        """
        Return the dimension of the code.
        """
        
        return rank(self.get_generator_matrix())

class GoppaEncoder(Encoder):
    r"""
    Encoder for Goppa codes
    
    INPUT:
    - ``code`` -- code associated with the encoder
    """
    def __init__(self, code):
        """
        Initialize.
        """
        super(GoppaEncoder, self).__init__(code)
        
    def _repr_(self):
        """
        Representation of a encoder for a Goppa code
        """
        return "Encoder for {}".format(self.code())
    
    def get_generator_matrix(self):
        """
        Return a generador matrix of the code
        """
        return self.code().get_generator_matrix()
    
    def encode (self, m):
        """
        Return a codeword
        
        INPUT:
        - ``m``: a vector to encode
        """
        return m * self.get_generator_matrix()
    
Goppa._registered_encoders["GoppaEncoder"] = GoppaEncoder

class GoppaDecoder(Decoder):
    r"""
    Decoder for Goppa codes
    
    INPUT:
    - ``code``: code associated with the decoder
    """
    
    def __init__(self, code):
        """
        Initialize
        """
        super(GoppaDecoder, self).__init__(code, code.ambient_space(), "GoppaDecoder")
                
        self._generating_pol = self.code().get_generating_pol()
        self._defining_set = self.code().get_defining_set()
    
    def _repr_(self):
        """
        Representation of a decoder for a Goppa code
        """
        return "Decoder for {}".format(self.code())
    
    def get_syndrome(self, c):
        """
        Return the syndrome polynomial
        
        INPUT:
        - ``c``: a element of the input space of ``self``.
        """
        field = self.code()._field
        field_L = self.code()._field_L
        
        embFL = FiniteFieldHomomorphism_generic(Hom(field,field_L))
        
        h = self.code().get_parity_pol()
        
        syndrome = 0
        
        for i in range(len(h)):    
            syndrome = syndrome + embFL(c[i])*h[i]
            
        return syndrome
    
    def get_generating_pol(self):
        """
        Return the generating polynomial
        """
        return self._generating_pol
    
    def decode_to_code(self, word):
        r"""
        Corrects the errors in ``word`` and returns a codeword.
        INPUT:
        - ``word`` -- a codeword of ``self``
        """
        i = 0
        field = self.code()._field
        field_L = self.code()._field_L
        
        embFL = FiniteFieldHomomorphism_generic(Hom(field,field_L))
        secLF = embFL.section()

        # Step 1
        S = self.get_syndrome(word)
        
        if S == 0:
            return word

        # Step 2
        r_prev = self.get_generating_pol()
        t = floor(self.get_generating_pol().degree()/2)
        r_i = S
        U_prev = 0
        U_i = 1
        
        # Steps 3 and 4
        while r_i.degree() >= t or r_prev.degree() < t:
            (q, r) = r_prev.quo_rem(r_i)
            aux_r_i = r_i
            aux_U_i = U_i
            r_i = r
            U_i = q * U_i + U_prev
            r_prev = aux_r_i
            U_prev = aux_U_i  
            i += 1
        
        # Step 5
        # make sigma monic     
        sigma = U_i / U_i.coefficients()[-1]
        
        eta = (-1)^i * r_i / U_i.coefficients()[-1]
        
        # roots of sigma are the locations of the errors
        roots_loc = []
        g = self.get_generating_pol()
        for root in sigma.roots():
            if g(root[0] != 0):
                roots_loc = roots_loc + [self._defining_set.index(root[0])]
                
        error = [0] * len(self._defining_set)
        x = self.get_generating_pol().parent().gen()
        sigma_diff = sigma.diff(x)
        
        for i in range(0, len(roots_loc)):
            error[roots_loc[i]] = eta.subs(x = sigma.roots()[i][0]) / (sigma_diff.subs(x = sigma.roots()[i][0]))
            
        x = word - vector(field, error)
        
        return x

    def decode_to_message(self, word):
        word = self.decode_to_code(word)
        return self.code().get_generator_matrix().solve_left(word)

Goppa._registered_decoders["GoppaDecoder"] = GoppaDecoder

# McEliece cryptosystem

In [4]:
def get_weight(c):
    w = 0
    
    for elem in c:
        if elem != 0:
            w += 1
    
    return w

In [5]:
class McEliece:
    r"""
    Implementation of McEliece cryptosystem.
    
    INPUT:
    - ``n`` -- 
    
    - ``defining_set`` -- tuple of n distinct elements of `\GF{p^m}`
    that are roots of `generating_pol
    """
    def __init__(self, n, q, g):
        """
        Initialize.
        """
        # Goppa code
        F = GF(2)
        L = GF(2^q)
        defining_set = get_defining_set(n, g, L)
        C = Goppa(defining_set, g, F)
        G = C.get_generator_matrix()
        k = G.nrows()
        n = G.ncols()
    
        # random binary nonsingular matrix
        S = matrix(F, k, [choice(F.list()) for i in range(k^2)])
        
        while rank(S) < k:
            i = randint(0, k-1)
            j = randint(0, k-1)
            S[i,j] = choice(F.list())
        
        # random permutation matrix
        columns = list(range(n))
        P = matrix(F, n)
        
        for i in range(n):
            l = randint(0, len(columns)-1)
            j = columns[l]
            P[i,j] = 1
            columns.remove(j)
        
        self._goppa = C
        self._G = G
        self._t = floor(g.degree()/2)
        self._k = k
        self._n = n
        self._S = S
        self._P = P
        self._public_key = S * self._G * P
        
    def __repr__(self):
        """
        Representation of a McEliece cryptosystem.
        """
        return "McEliece cryptosystem over {}".format(self._goppa)
        
    def get_S(self):
        return self._S
    
    def get_G(self):
        return self._G
    
    def get_P(self):
        return self._P
    
    def get_public_key(self):
        return self._public_key

    def encrypt(self, m):
        """
        Return a chipertext
        
        INPUT:
        - ``m``: a plaintext to encrypt
        """
        # random errors
        e = vector([F(0) for i in range(self._n)])
        
        while get_weight(e) != self._t:
            i = randint(0, len(e) - 1)
            e[i] = choice(F.list())
        
        # chipertext
        c = m * self._public_key + e
        
        print("Error: " + str(e))
        
        return c
    
    def decrypt(self, c):
        word = c * self._P^(-1)
        D = GoppaDecoder(self._goppa)
        word = D.decode_to_message(word)
        message = word * (self._S)^(-1)
        
        return message

#### Example 1

In [40]:
q = 8
n = 60
t = 4
F = GF(2)
L = GF(2^q)
a = L.gen()
R.<x> = L[]

while True:
    g = R.random_element(t)
    
    if g.is_monic():
        break
g

x^4 + (z8^6 + z8^5 + z8^4 + z8^3 + z8^2 + z8 + 1)*x^3 + (z8^6 + z8^4 + z8^3 + z8^2 + z8 + 1)*x^2 + (z8^7 + z8^5 + z8 + 1)*x + z8^5 + z8^4 + z8^2 + 1

In [41]:
ME = McEliece(n, q, g)

print("Matriz S:")
show(ME.get_S())

print("Matriz P:")
show(ME.get_P())

print("Matriz G' (clave pública):")
show(ME.get_public_key())

Matriz S:


28 x 28 dense matrix over Finite Field of size 2 (use the '.str()' method to see the entries)

Matriz P:


60 x 60 dense matrix over Finite Field of size 2 (use the '.str()' method to see the entries)

Matriz G' (clave pública):


28 x 60 dense matrix over Finite Field of size 2 (use the '.str()' method to see the entries)

In [8]:
public_key = ME.get_public_key()
message = []
while len(message) != public_key.nrows():
    message = message + [choice(F.list())]

message = vector(F, message)

print("\nMessage to encrypt:")
print(message)

encrypted_message = ME.encrypt(message)

print("\nEncrypted message:")
print(encrypted_message)


Message to encrypt:
(1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0)
Error: (0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)

Encrypted message:
(0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0)


In [9]:
message2 = ME.decrypt(encrypted_message)
message2

(1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0)

In [10]:
message == message2

True

#### Example 2

In [32]:
q = 5
n = 30
t = 4
F = GF(2)
L = GF(2^q)
a = L.gen()
R.<x> = L[]

while True:
    g = R.random_element(t)
    
    if g.is_monic():
        break
g

x^4 + (z5^2 + z5 + 1)*x^3 + (z5^4 + z5^3 + z5^2 + z5)*x^2 + (z5^4 + z5^3 + z5^2)*x + z5^2 + z5

In [33]:
McEliece(n, q, g)

McEliece cryptosystem over [30, 10] Goppa code

In [34]:
ME = McEliece(n, q, g)

print("Matriz S:")
show(ME.get_S())

print("Matriz P:")
show(ME.get_P())

print("Matriz G' (clave pública):")
show(ME.get_public_key())

Matriz S:


[1 0 1 1 1 1 1 1 0 0]
[1 1 0 1 1 1 1 1 1 0]
[1 1 0 1 0 1 0 0 1 0]
[1 0 1 1 0 1 1 1 1 0]
[1 1 1 0 1 0 1 1 0 1]
[0 0 1 0 0 0 1 1 1 1]
[0 0 1 1 0 0 1 0 0 1]
[0 0 1 1 0 1 0 0 1 0]
[1 0 1 1 1 1 1 1 1 1]
[0 0 1 0 0 1 0 0 0 1]

Matriz P:


30 x 30 dense matrix over Finite Field of size 2 (use the '.str()' method to see the entries)

Matriz G' (clave pública):
[1 1 0 0 0 0 1 0 0 0 1 1 1 0 1 0 1 1 1 0 1 1 0 0 1 0 0 1 0 1]
[1 1 0 1 0 1 1 1 0 1 1 1 1 1 1 1 1 0 1 0 0 1 1 1 1 0 1 1 0 1]
[0 0 1 0 0 0 0 1 1 1 0 1 1 1 1 1 0 1 0 0 0 0 1 1 0 0 0 1 0 0]
[0 0 1 0 1 1 0 0 1 1 1 1 1 0 1 1 1 1 0 0 1 1 0 0 0 0 1 0 1 0]
[1 1 0 0 1 1 0 0 0 0 1 1 0 1 0 0 1 0 0 1 1 1 0 1 0 1 1 0 1 1]
[0 0 0 1 0 0 0 1 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 1 1 1 0 1 0 0]
[0 1 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 1 0 1 1 1 1 0 0]
[1 1 0 0 0 1 1 1 0 1 0 0 1 0 1 1 1 0 1 1 1 0 0 0 1 0 1 0 1 0]
[0 1 0 0 0 0 0 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 0 0 1 0 0 0 1]
[0 1 0 1 0 1 0 0 0 0 0 0 0 0 1 1 0 0 1 0 1 0 1 0 0 1 0 0 1 0]


[1 1 0 0 0 0 1 0 0 0 1 1 1 0 1 0 1 1 1 0 1 1 0 0 1 0 0 1 0 1]
[1 1 0 1 0 1 1 1 0 1 1 1 1 1 1 1 1 0 1 0 0 1 1 1 1 0 1 1 0 1]
[0 0 1 0 0 0 0 1 1 1 0 1 1 1 1 1 0 1 0 0 0 0 1 1 0 0 0 1 0 0]
[0 0 1 0 1 1 0 0 1 1 1 1 1 0 1 1 1 1 0 0 1 1 0 0 0 0 1 0 1 0]
[1 1 0 0 1 1 0 0 0 0 1 1 0 1 0 0 1 0 0 1 1 1 0 1 0 1 1 0 1 1]
[0 0 0 1 0 0 0 1 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 1 1 1 0 1 0 0]
[0 1 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 1 0 1 1 1 1 0 0]
[1 1 0 0 0 1 1 1 0 1 0 0 1 0 1 1 1 0 1 1 1 0 0 0 1 0 1 0 1 0]
[0 1 0 0 0 0 0 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 0 0 1 0 0 0 1]
[0 1 0 1 0 1 0 0 0 0 0 0 0 0 1 1 0 0 1 0 1 0 1 0 0 1 0 0 1 0]

In [35]:
# Encrypt
public_key = ME.get_public_key()
message = []
while len(message) != public_key.nrows():
    message = message + [choice(F.list())]

message = vector(F, message)

print("\nMessage to encrypt:")
print(message)

encrypted_message = ME.encrypt(message)

print("\nEncrypted message:")
print(encrypted_message)


Message to encrypt:
(0, 1, 0, 0, 0, 0, 0, 0, 1, 1)
Error: (0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)

Encrypted message:
(1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0)


In [36]:
message2 = ME.decrypt(encrypted_message)
message2

(0, 1, 0, 0, 0, 0, 0, 0, 1, 1)

In [15]:
message == message2

True

# Temporal

In [16]:
L = GF(2^5)
a = L.gen()
R.<x> = L[]
g = x^3 + (a^4 + a^3 + 1)*x^2 + (a^4 + 1)*x + a^4 + a^2 + a + 1
n = 20
McEliece(n, q, g)

McEliece cryptosystem over [20, 5] Goppa code

In [17]:
print(ME.get_P())

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]


In [18]:
print(ME.get_public_key())

[0 1 1 1 1 1 1 1 1 0 0 1 0 0 1 0 0 1 0 1]
[1 1 0 1 0 0 1 1 0 1 0 0 1 0 0 1 1 1 1 0]
[1 0 0 1 0 0 0 0 1 0 0 1 1 0 0 1 0 1 1 1]
[0 0 0 1 0 0 1 1 1 1 1 0 0 1 0 1 0 0 1 0]
[0 0 1 1 0 1 0 1 0 1 1 0 0 0 0 1 1 0 1 1]


In [19]:
message = vector(GF(2), (1, 1, 0, 0, 0))
ME.encrypt(message)

Error: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0)


(1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1)

In [20]:
encrypted_message = vector(GF(2), (1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0))

In [21]:
ME.decrypt(encrypted_message)

ValueError: matrix equation has no solutions

In [ ]:
v = vector(GF(2), (1, 1, 0, 1, 0))
get_weight(v)